<a href="https://colab.research.google.com/github/lijingwang/Bayesian_inversion/blob/main/sgsim_anisotropy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
from numpy.fft import fftn, fftshift, ifftn
from numpy.random import uniform as uniform
from scipy.stats import norm
from numpy.core.umath_tests import inner1d

def unconditional_simu(v, mu, sill, rangeX, rangeY, rangeZ,  model, nX, nY, nZ):
    '''
    Unconditional Simulation with different variogram

    Args: 
        nX, nY, nZ: (int) number of pixels you want for each dimension, if it is 2D, nZ = 1
        mu:         (float) mean
        sill:       (float) variance
        rangeX, rangeY, rangeZ: (float) Range/correlation length for each dimension, >0
        model:      (str) variogram model, Spherical/Gaussian
    '''

    xx, yy, zz = np.meshgrid(np.arange(nX), np.arange(nY), np.arange(nZ))
    points = np.stack((xx.ravel(), yy.ravel(), zz.ravel())).T

    centroid = points.mean(axis=0)
    range_list = np.array([rangeX, rangeY, rangeZ])
    h = np.linalg.norm((points - centroid) / range_list, axis = 1).reshape((nY, nX, nZ))

    if model == 'Exponential':
        c = np.exp(-3*h) * sill
    elif model == 'Gaussian':
        c = np.exp(-3*(h)**2) * sill

    grid = fftn(fftshift(c)) / (nX * nY * nZ)
    grid = np.abs(grid)
    grid[0, 0, 0] = 0 # reference level
    ran = np.sqrt(grid) * np.exp(1j * np.angle(fftn(np.swapaxes(v,0,1))))
    grid = np.real(ifftn(ran * nX * nY * nZ))
    std = np.std(grid)

    if nX == 1 or nY == 1 or nZ == 1: 
        grid = np.squeeze(grid)
    
    grid = np.swapaxes(grid,0,1)
        
    return grid / std * np.sqrt(sill) + mu